In [1]:
# Date: Dec 6, 2022
# Authors: Sashka Warner, Alison Errico, David Tang
# Desc: Clean UWWI services dataset for use in joining services with sites

# Import libraries
import pandas as pd
import re

In [2]:
# Load Service data from csv
services_raw = pd.read_csv('../../uwwi_datasets/uwwi_dataset_services_encoded.csv')

In [3]:
# Inspect the data
services_raw.head()

,Agency_Id,Site_Id,Service_Id,Service_CreateStamp,Service_EditStamp,Service_Status,ServiceSystem_Active,ServiceSystem_Name,ServiceSystem_LanguagesOtherThanEnglishText,ServiceSystem_DocumentsRequired,...,ServiceOption_Aoda_Residential - State Certified,ServiceOption_Aoda_Outpatient Services,ServiceOption_Aoda_Inpatient Services,ServiceOption_Aoda_Detox Services,ServiceOption_Aoda_MAT Treatment,ServiceOption_Aoda_AODA Web General,ServiceOption_Aoda_County Crisis Lines,ServiceOption_Aoda_County Health Services,ServiceOption_Aoda_Approved Peer Run Recovery Organization,ServiceOption_Aoda_Drug Take Back Locations (Do Not Use)
0,1,414.0,3430.0,2017-03-16T16:07:41Z,2022-01-05T16:30:04Z,active,No,ZZZ HOUSING - INTEGRATED COMMUNITY SOLUTIONS,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,414.0,6309.0,2017-03-16T16:13:48Z,2022-05-12T17:51:04Z,active,Yes,HOUSING - HOMEOWNERSHIP,"English, Spanish",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,414.0,6310.0,2017-03-16T16:13:48Z,2022-05-12T17:51:40Z,active,Yes,HOUSING - HOUSING CHOICE VOUCHER PROGRAM,"English, Spanish, Hmong. (Spanish language ap...",Completed application\nProof of residency in B...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,414.0,6311.0,2017-03-16T16:13:49Z,2022-05-12T17:52:46Z,active,Yes,LIFE SKILLS - FAMILY SELF-SUFFICIENCY,"English, Spanish",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,414.0,64344.0,2021-11-03T20:13:11Z,2022-10-04T14:06:03Z,active,Yes,LADDER OF HOPE HOLIDAY ASSISTANCE,NaN,Interest Form,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Inspect type of time stamp fields
#type(services_raw["Service_CreateStamp"][0])
#type(services_raw["Service_EditStamp"][0])

In [37]:
# Make a copy of the data
services = services_raw.copy()

# drop sites with no Service_Id
services = services.dropna(subset = ['Service_Id']) 
services.reset_index(drop=True, inplace=True)

# Convert time stamps to date time format (takes a long time, which is why there are separate scripts for each dataset)
#services["Service_CreateStamp"] = pd.to_datetime(services["Service_CreateStamp"])

# Continue converting time stamps to date time format
#services["Service_EditStamp"] = pd.to_datetime(services["Service_EditStamp"])

# Use ServiceOption_ cols to filter by service type
# ServiceOption__NA = 1 implies that there is no service offered of  type
# ServiceOption__NA = NA implies that there is at least one service offered of the  type
# Change NA to 0 for ease
# For sanity, remove __NA from column names, and get the opposite of the values 
# such that 1 means the( site does have the service, and 0 means the site does not have the service

# Get col names containing 'NA'
service_col_names = [col for col in services.columns if 'NA' in col]

# Filter df for these columns
service_cols = services[service_col_names]
# Replace NAs with 0
service_cols = service_cols.fillna(0)
# Get opposite of services values (eg swap 0 with 1, and 1 with 0)
service_cols_opposite = service_cols.applymap(lambda x: float(not x))

# Get ID columns
id_col_names = ["Service_Id", "Site_Id"]
id_cols = services[id_col_names]
id_cols.reset_index(drop=True, inplace=True)
# Get selected column names
id_col_names.extend(service_col_names)

# Combine the data with the service and site id 
services = pd.concat(
    objs=[id_cols, service_cols_opposite], 
    axis=1, 
    names = id_col_names)

# Remove _NA and prefix in column names
services.columns = [re.sub("_NA|ServiceOption_", "", s) for s in services.columns]

#Change Days to DaysOptions to avoid overwriting reserved keyword
services.rename(columns={"Days": "DaysOption"}, inplace=True)

# Inspect result
services.tail()



,Service_Id,Site_Id,PaymentOptions,PopulationFocus,LanguagesOtherThanEnglishNew,DaysOption,MentalHealthConditions,MedicalConditions,CounselingTypesOffered,AgeGroup,TherapyAndSupportiveApproaches,IssueOfFocus,Aoda
40087,83394.0,48178.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
40088,83395.0,48178.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
40089,83398.0,48182.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
40090,83400.0,48182.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
40091,83403.0,48183.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
# Save output to csv in a `clean/` directory
# Specify index=False so that columns match DB
# Specify header=False to match snowflake input requirements
services.to_csv("../data/uwwi_dataset_services_cleaned.csv", index=False, header=False)

In [39]:
# Check types
#[type(x) for x in services.iloc[0,:]]
[x.upper() for x in services.columns]

['SERVICE_ID',
 'SITE_ID',
 'PAYMENTOPTIONS',
 'POPULATIONFOCUS',
 'LANGUAGESOTHERTHANENGLISHNEW',
 'DAYSOPTION',
 'MENTALHEALTHCONDITIONS',
 'MEDICALCONDITIONS',
 'COUNSELINGTYPESOFFERED',
 'AGEGROUP',
 'THERAPYANDSUPPORTIVEAPPROACHES',
 'ISSUEOFFOCUS',
 'AODA']